<!-- Python notebook on basic linear control: negative feedback loops, feedforward terms, PID control, LQR and basic Lyapunov analysis. -->

# Robot control systems

Robots are equipped with various actuators that receive inputs and sensors that provide outputs. A robot's brain (its computer) faces the task of deciding what signals to send to the actuators based on the measured sensor readings such that a robot executes desired behavior. This task of controlling the robot must address various challenges such as measurement noise, inaccurate robot models, non-ideal actuators and  external disturbances. Control theory offers a systematic approach to address this challenge by designing control systems to manage robot behavior. 


<img src="tp3/media/system.png" alt="A system" width="500"/>



A robot can be modelled as a _system_ taking inputs $u(t)$ and producing outputs $y(t)$, such that $y(t) = h(u(t))$. 

Let us start with a simple, yet widely-applicable class of systmes, namely a discrete-time linear time invariant (LTI) system. It can be expressed the the so-called state-space form as follows

$x_{k+1} = Ax_k + Bu_k$,

and the output is also given by a linear equation

$y_k = Cx_k + Du_k$.

The matrix $D = 0$ most often in practice. The system is called linear because the system dynamics is linear in $x$ and $u$, the input-output relationship is affine, and time-invariant because the dynamics parameters $A, B, C$, and $D$ are assumed to not depend on time.

#### Exercise 3.1: 

Consider a point-mass object of mass $m$ is free to move along x-axis. The only force acting on it is the external force $f$ in the x-direction. 

$m\ddot{x} = f$

Consider the position and velocity of the object as the states and the $f$ as the control input, x-position as the output. Use forward Euler integration scheme with timestep $T_s$ for discretizing the system. Provide the $A,B,C,$ and $D$ matrices associated with this system.

Answer: Add here.

## Trajectory tracking

Consider the trajectory-tracking task, where the system should controlled such that its output follows a desired trajectory $[y_{\mathrm{des},0}, y_{\mathrm{des},1}, y_{\mathrm{des},2}, y_{\mathrm{des},3}, ... ]$. One of the simplest controllers that can try to execute this task is an `open-loop' feedforward controller shown below.

<img src="tp3/media/FF.png" alt="Feedforward control system" width="500"/>

Here, the feedforward controller computes the appropriate control input to apply depending on the desired system output. It uses knowledge of the starting state of the system and inverts the system dynamics to compute the trajectory of control inputs to be applied. For example, for a system at state $x_k$ the linear equation below can be solved for $u_k$ to compute the feedforward control action for trajectory following. 

$y_{\mathrm{des},k+1} = C(Ax_k + Bu_k) + Du_k$.

Pros of pure feedforward control: 
- FF control systems are simple.
- No sensing and estimation are required.
- Can follow complex desired trajectories.

Cons of pure feedforward control:
- Rarely works in practice.
- Sensitive to initial state estimation error and model errors.
- Inverting system dynamics may not always be possible.

### Exercise 3.2:
For the point mass system derived in exercise 3.1 and the desired x-positions given below, compute the feedforward control actions. (Hint: Dynamics not directly invertible for tracking position, try tracking the velocity.)

In [ ]:
import pinocchio as pin
from utils.meshcat_viewer_wrapper import MeshcatVisualizer
import time
import numpy as np
from numpy.linalg import inv,norm,pinv,svd,eig
from scipy.optimize import fmin_bfgs,fmin_slsqp
from scipy.linalg import solve_discrete_are
from utils.load_ur5_with_obstacles import load_ur5_with_obstacles,Target
import example_robot_data as robex
import matplotlib.pylab as plt

In [ ]:
np.random.seed(0)
# track a sine function
frequency = 2
Ts = 0.01
m = 1

y_des = np.array([np.sin(2*np.pi*frequency*i*Ts) for i in range(200)])

# define an LTI system class
class LTI_system:
    def __init__(self,A,B,C=None,D=None):
        self.A = A
        self.B = B
        if C is None:
            self.C = np.eye(A.shape)
        else:
            self.C = C
        if D is None:
            self.D = np.zeros((C.shape[0],B.shape[1]))
        else:
            self.D = D
        
    def step(self,x,u):
        return self.A @ x + self.B @ u
    
    def output(self,x,u):
        return self.C @ x + self.D @ u
    
    def simulate(self,x0,u_traj):
        x_step = self.step(x0,u_traj)
        y_step = self.output(x0,u_traj)
        return x_step, y_step

u_traj_ff = []
y_traj = []
x_traj = []
x0 = np.zeros((2,))
x_traj.append(x0)

## Your code goes below
## Use the LTI class above to initialize point-mass system from Ex 3.1, compute the feedforward actions and simulate the system. 

In [ ]:
# plot the simulated trajectory and the desired trajectory
plt.figure()
plt.plot(y_des)
plt.plot(y_traj)
plt.legend(['desired','simulated'])
plt.xlabel('time (s)')
plt.ylabel('position (m)')

plt.show()

### Exercise 3.3: 
Suppose that the mass of the system is not exactly known and that the true mass is 0.9 times the previous mass. Create a new system with this true mass and compute y_traj_real.

In [ ]:
y_traj_real = []
m_new = 0.9

#Your code here


In [ ]:
plt.figure()
plt.plot(y_des)
plt.plot(y_traj_real)
plt.plot(y_traj)
plt.legend(['desired','real','feedforward'])
plt.xlabel('time (s)')
plt.ylabel('y (m)')
plt.show()

## Feedback control

It is clear from the previous plot that the feedforward control is not enough to track the desired trajectory in the presence of noise, model errors and disturbances. We need a negative feedback loop that measures the error between the desired trajectory and the real trajectory and uses it to reduce tracking errors. Negative feedback loops, being widely and frequently used, are nearly synonymous with the field of control.

<img src="tp3/media/FB.png" alt="Feedback control system" width="500"/>

For LTI systems, a common form of negative feedback is the linear state feedback law 

$u = K (x_\mathrm{des} - x)$.

If the control task involves taking the system to the origin ($x_\mathrm{des} = 0$),the closed-loop system is then described by the following discrete-time LTI system:

$x_{k+1} = (A - BK)x_k$

$y_k = Cx_k$

### Exercise 3.4
For the point-mass system, design a state feedback controller for tracking the desired trajectory. Use zero for the desired velocity. (Hint: The design involves guessing the matrix K.)

Additional info: We are implementing a proportional-derivative (PD) controller. The first term of K acting on the position error, is the proportional controller gain, while the second term of K, acting on the velocity error, is the derivative error controller gain.

In [ ]:
y_des.shape
x_des = np.vstack([y_des,y_des*0])

In [ ]:
y_traj_fb = []
x_traj_fb = []
x_traj_fb.append(x0)

x_des = np.vstack([y_des,y_des*0])

K = np.array([[]]) # Add your feedback gains here

u_traj_fb = []

for i in range(200):

    pass
    # Add your code here
    # u = ?
    u_traj_fb.append(u)
    # x_step, y_step = sys.simulate(x_traj_fb[-1], u)



In [ ]:
# Plot state tracking
plt.figure()
plt.plot(y_des)
plt.plot(y_traj_fb)
plt.legend(['desired','simulated'])
plt.xlabel('time (s)')
plt.ylabel('position (m)')
plt.show()

## Exercise 3.5
Suppose that the system also has noisy dynamics. Add Gaussian noise to the x_step output of the LTI system. Simulate the system you designed in Ex 3.4. What can you say about its behavior and how the magnitude of $K$ affects tracking in noisy systems.

Superior tracking often requires both feedforward and feedback control as shown in 

<img src="tp3/media/FF_FB.png" alt="Feedback control system" width="700"/>

Feedforward terms permit tracking complex trajectories using the knowledge of system dynamics while the feedback terms stabilize the system around the desired setpoint.

### Exercise 3.6
Combine the feedback and feedforward actions to track the desired trajectory for the point mass system from Exercise 3.3.

In [ ]:
y_traj_fbff = []
x_traj_fbff = []
u_traj = []
x_traj_fbff.append(x0)

for i in range(100):

    pass


In [ ]:
# plot the simulated trajectory and the desired trajectory

plt.figure()
plt.plot(y_des)
plt.plot(y_traj_fbff)
plt.legend(['desired','simulated'])
plt.xlabel('time (s)')
plt.ylabel('position (m)')
plt.show()

## System stability

Let us consider regulation problem, where the desired trajectory being tracked is a fixed set-point. Without loss of generality (why?) for regulation problems, we can assume that we want the system to reach the origin. 

It is important to characterize the stability of the feedback system, as it is a property that implies 'good behavior' of the closed-loop system, i.e. that it will not diverge or oscillate from the desired set-point.  Unstable systems can be dangerous in practice. 

The stability of closed-loop LTI systems can be straightforwardly characterized using the eigenvalues of the closed-loop system matrix $A_\mathrm{cl} = A - B K$. The magnitude of all the eigenvalues of the matrix A - B*K should be less than 1.0. For LTI systems, this is equivalent to checking that the system is strictly stable. If one eigenvalue has a magnitude equal to 1, the system is said to be marginally stable. If more than one eigenvalue has a magnitude equal to 1, the system may be unstable. If even one eigenvalue magnitude is strictly greater than one, the system is unstable. Note that since 

$x_{k+1} = A_\mathrm{cl} x_k$,

strict stability implies that the state converges to the origin at an exponential rate (exponential rate is not so fast when you are decreasing compared to when you are increasing).

### Exercise 3.7 

Compute the eigenvalues of the point-mass system and the closed-loop system with the linear feedback gains chosen in Exercise 3.4 and characterize the stability property of these two systems.

Answer: Enter your answer here.

## Linear Quadratic Regulator

While it was fairly straightforward to guess stable feedback gains for the point mass system, this is not so easy for more complex systems with higher dimensions. Fortunately, there exist several methods to compute these gains. One of the most popular methods is Linear Quadratic Regulator (LQR). LQR is a method that minimizes the cost function, (assuming that the system is being stabilized around the origin):

$J = \sum_{t=0}^{\infty} (x_t^T Q x_t + u_t^T R u_t)$,
where $Q$ and $R$ are positive definite matrices. The LQR controller is the optimal feedback gain that minimizes this cost function. The LQR controller can be computed using the following steps:

 1. Solve the discrete-time Riccati equation for the system:
 
    $ P = A^T P A - A^T P B (B^T P B + R)^{-1} B^T P A + Q $ 

2. Compute feedback gain

    $K = (R + B^T P B)^{-1} B^T P A$

3. The control law is then:
    
    $ u = -K x $

LQR (and linear control in general) is a useful tool even for controlling nonlinear systems. Nonlinear systems are often locally linearizable, using their first derivatives. The controller design is then performed for this linearized system and can be effective as long as the nonlinear system remains close to the linearization point.

In this exercise, we will consider the nonlinear system cart-pole, linearized around the upright position. The state of the system is 

$x = \begin{bmatrix} x \\ \theta \\ \dot{x} \\ \dot{\theta} \end{bmatrix}$.

The system dynamics and the linearization around the upright position can be found in 
https://courses.ece.ucsb.edu/ECE594/594D_W10Byl/hw/cartpole_eom.pdf

Assume that $M = 0$, $m_p = 1$ and $L = 1$.

### Exercise 3.8

Compute the LQR feedback gains for the cart-pole system linearized at the upright position. Stabilize the system at the upright position the LQR controller, starting from a state close to the upright position. Simulate the nonlinear system for 10 seconds and plot the state trajectories.

(Hint: use scipy.linalg.solve_discrete_are to solve the Riccati equation)

## Bonus question (coming)

Let us setup a robot system that takes joint velocities as inputs and provides EE positions as the output.

In [ ]:
class Robot:

    def __init__(self, robotname, q0, ee_name, Ts = 0.01):
        self.robot = robex.load(robotname)
        self.x = q0
        self.ee_id = self.robot.model.getFrameId(ee_name)
        self.Ts = Ts

    def fk_ee(self, q = None):
        if q is None:
            q = self.x
        robot = self.robot
        pin.framesForwardKinematics(robot.model, robot.data, q)
        return self.robot.data.oMf[self.ee_id]
    
    def apply_input(self, u, noise = 'normal'):
        if noise == 'normal':
            u += np.random.normal(0, 0.1, 6)*self.Ts
        self.x = pin.integrate(self.robot.model, self.x, u*self.Ts)
        
        return self.fk_ee(self.x)
    

q0 = np.array([0, -1.5, 1.5, 0, 0, 0])
robot = Robot('ur5', q0, 'tool0')

viz = MeshcatVisualizer(robot.robot)
viz.display(robot.x)

In [ ]:
robot.fk_ee()
# len(robot.robot.data.)
# robot.ee_id

Let $$p(t) = \mathrm{fk_{ee}}(q(t))$$ be the end-effector position at time $t$.